In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from pandas import Series
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor, RandomForestRegressor

from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc, roc_auc_score, roc_curve
RANDOM_SEED = 42

Setup

In [ ]:
VERSION    = 1
DIR_TRAIN  = '../input/parsing-all-moscow-auto-ru-09-09-2020/'
DIR_TEST   = '../input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.20   # 20%
#VAL_SIZE   = 0.33   # 33%
N_FOLDS    = 5
#CATBOOST
ITERATIONS = 1000
LR         = 0.1

In [ ]:
train = pd.read_csv(DIR_TRAIN+'all_auto_ru_09_09_2020.csv') # dataset taken for training the model
test = pd.read_csv(DIR_TEST+'test.csv')
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

In [ ]:
train.info()

In [ ]:
test.info()

**Data Preprocessing**

In [ ]:
# first we got to unify the columns in train and test datasets

In [ ]:
# test['model_name'] equals  train['model']
test = test.rename(columns = {'model_name':'model'})

In [ ]:
#deleting rows without price
train.dropna(subset=['price'], inplace=True)

In [ ]:
# got to check the common columns between train and test
print(len(train.columns), len(test.columns))

In [ ]:
A = list(test.columns)
B = list(train.columns)
Intersection_A_B = list(set.intersection(set(A), set(B)))

In [ ]:
# list of the common columns in train and set
Intersection_A_B

In [ ]:
test = test[Intersection_A_B]

In [ ]:
Intersection_A_B.append('price')

In [ ]:
train = train[Intersection_A_B]

In [ ]:
train['sample'] = 1 # labelling train
test['sample'] = 0 # labelling test

data = test.append(train, sort=False).reset_index(drop=True)

In [ ]:
data.info()

BodyType

In [ ]:
data.bodyType.value_counts()

In [ ]:
data.bodyType = data.bodyType.astype(str).apply(lambda x: None if x.strip()=='' else x)
data.bodyType = data.bodyType.apply(lambda x: x.split(' ')[0].lower())

In [ ]:
data.bodyType = data.bodyType.replace('nan', 'внедорожник')

Color

In [ ]:
data.color.value_counts()

In [ ]:
color_dict = {'CACECB': 'серебристый', 
              'FAFBFB':'белый', 
              'EE1D19':'красный', 
              '97948F':'серый',
              '660099':'пурпурный',
              '040001':'чёрный',
              '4A2197':'фиолетовый',
              '200204':'коричневый',
              '0000CC':'синий',
              '007F00':'зелёный',
              'C49648':'бежевый',
              '22A0F8':'голубой',
              'DEA522':'золотистый',
              'FFD600': 'жёлтый',
              'FF8649':'оранжевый',
              'FFC0CB':'розовый',
              'серебристый': 'серебристый', 
              'белый':'белый', 
              'красный':'красный', 
              'серый':'серый',
              'пурпурный':'пурпурный',
              'чёрный':'чёрный',
              'фиолетовый':'фиолетовый',
              'коричневый':'коричневый',
              'синий':'синий',
              'зелёный':'зелёный',
              'бежевый':'бежевый',
              'голубой':'голубой',
              'золотистый':'золотистый',
              'жёлтый': 'жёлтый',
              'оранжевый':'оранжевый',
              'розовый':'розовый'}

In [ ]:
data.color = data.color.map(color_dict)

ModelDate|ProductionDate

In [ ]:
data.modelDate.value_counts()

In [ ]:
data.modelDate = data.modelDate.fillna('2010')
data.modelDate = data.modelDate.astype('int')

In [ ]:
# to make it closer to a real estimation of the car, changing the logic of a columns:
data.modelDate = 2021 - data.modelDate # model's age
data.productionDate = 2021 - data.productionDate #vehicle's age

NumberOfDoors

In [ ]:
data.numberOfDoors.value_counts()

In [ ]:
data.numberOfDoors = data.numberOfDoors.fillna(5.0)

In [ ]:
data.numberOfDoors = data.numberOfDoors.astype(int)

vehicleConfiguration|vehicleTransmission

In [ ]:
data.vehicleConfiguration.value_counts()

In [ ]:
data.vehicleConfiguration = data.vehicleConfiguration.fillna('SEDAN MECHANICAL 1.6')
#taking the second word from the column we set the gearbox type of the vehicle
data.vehicleConfiguration = data.vehicleConfiguration.astype(str).apply(lambda x: None if x.strip()=='' else x)
data.vehicleConfiguration = data.vehicleConfiguration.apply(lambda x: x.split()[1])

In [ ]:
data.vehicleTransmission.value_counts()

In [ ]:
# the same data as in vehicleConfiguration column, dropping
data = data.drop('vehicleTransmission', axis = 1)

engineDisplacement

In [ ]:
data.engineDisplacement.value_counts()

In [ ]:
data.engineDisplacement = data.engineDisplacement.astype(str).apply(lambda x: None if x.strip()=='' else x)
data.engineDisplacement = data.engineDisplacement.apply(lambda x: x[:3])
data.engineDisplacement = data.engineDisplacement.apply(lambda x: '2.0' if len(x)==1 else x )

In [ ]:
#changing the incorrect values to the most popular in dataset
data.engineDisplacement = data.engineDisplacement.apply(lambda x: '2.0' if x[1]!= '.' else x )

enginePower

In [ ]:
data.enginePower.value_counts()

In [ ]:
data.enginePower = data.enginePower.astype(str).apply(lambda x: None if x.strip()=='' else x)
data.enginePower = data.enginePower.apply(lambda x: x.split()[0])
data.enginePower = data.enginePower.astype(float)

In [ ]:
# func to set the catogories of the engine's power:
def engine_power(x):
    if x < 100: x = 1
    elif 99 < x < 150: x = 2
    elif 149 < x < 200: x = 3
    elif 199 < x < 250: x = 4
    elif 249 < x < 300: x = 5
    elif 299 < x < 350: x = 6
    elif 349 < x < 400: x = 7
    elif 399 < x < 450: x = 8
    elif 449 < x < 500: x = 9
    elif 499 < x < 550: x = 10
    elif 549 < x < 600: x = 11
    elif 599 < x: x = 12
    return x  
data.enginePower = data.enginePower.map(engine_power)

In [ ]:
data.enginePower = data.enginePower.fillna(2.0)
data.enginePower = data.enginePower.astype(int)

Mileage

In [ ]:
data.mileage.describe()

In [ ]:
def mile_age(x):
    if x < 20000: x = 1
    elif 19999 < x < 40000: x = 2
    elif 39999 < x < 60000: x = 3
    elif 59999 < x < 80000: x = 4
    elif 79999 < x < 100000: x = 5
    elif 99999 < x < 200000: x = 6
    elif 199999 < x < 500000: x = 7
    elif 499999 < x: x = 8
    return x  
data.mileage = data.mileage.map(mile_age)

In [ ]:
data.mileage.value_counts()

Привод

In [ ]:
data.Привод.value_counts()

In [ ]:
def drive(x):
    if x =='передний': x = 'front'
    elif x =='задний': x = 'rear'
    elif x=='полный': x = 'full'
    return x
data['Drive'] = data.Привод.map(drive)

In [ ]:
data = data.drop('Привод', axis = 1)
data.Drive = data.Drive.fillna('front')

In [ ]:
data.Drive.value_counts()

Руль|Steering

In [ ]:
data.Руль.value_counts()

In [ ]:
wheel = {'LEFT': 'LEFT', 
              'Левый':'LEFT', 
              'RIGHT':'RIGHT', 
              'Правый':'RIGHT',

             }
data['Steering'] = data.Руль.map(wheel)
data.Steering = data.Steering.fillna('LEFT')

In [ ]:
data.Steering.value_counts()

In [ ]:
data = data.drop('Руль', axis = 1)

Владельцы|Owners

In [ ]:
data.Владельцы.value_counts()

In [ ]:
data.Владельцы = data.Владельцы.astype(str).apply(lambda x: None if x.strip()=='' else x)

In [ ]:
owner = {'3.0': '3.0','2.0':'2.0','1.0':'1.0', 
        '3 или более':'3.0',
        '1 владелец':'1.0',
        '2 владельца':'2.0',
         }
data['Owners'] = data.Владельцы.map(owner)

In [ ]:
data['Owners'] = data['Owners'].fillna(3.0)

In [ ]:
data.Owners.value_counts()

In [ ]:
data['Owners'] = data['Owners'].apply(lambda x: int(x[0]) if type(x)==str else x)

In [ ]:
data.Owners = data.Owners.astype(int)
data = data.drop('Владельцы', axis = 1)

ПТС|Tech_passp

In [ ]:
data.ПТС.value_counts()

In [ ]:
tech_passp = {'ORIGINAL': 'ORIGINAL', 
              'Оригинал':'ORIGINAL', 
              'DUPLICATE':'DUPLICATE', 
              'Дубликат':'DUPLICATE',

             }
data['Tech_passp'] = data.ПТС.map(tech_passp)
data['Tech_passp'] = data['Tech_passp'].fillna('ORIGINAL')

In [ ]:
data.Tech_passp.value_counts()

In [ ]:
data = data.drop('ПТС', axis = 1)

In [ ]:
data.info()

In [ ]:
data.name.value_counts()

In [ ]:
data = data.drop(['Таможня', 'Состояние', 'Владение','description','name'], axis = 1)

In [ ]:
Label Encoding

In [ ]:
bin_cols = ['Steering','Tech_passp']
cat_cols = ['brand','bodyType','color','fuelType','numberOfDoors','vehicleConfiguration', 'engineDisplacement',
            'enginePower','mileage','Drive','Owners','model']
num_cols = ['modelDate','productionDate']

In [ ]:
label_encoder = LabelEncoder()
for column in bin_cols:
    data[column] = label_encoder.fit_transform(data[column])

In [ ]:
# separating dataset for  training and submission
data_train = data[data['sample']==1]
data_test = data[data['sample']==0]

In [ ]:
data_train.info()

In [ ]:
data_train = data_train.dropna(axis=0, subset=['price'])
data_train = data.drop('sample', axis = 1)

In [ ]:
test = pd.read_csv(DIR_TEST+'test.csv')
data_test['sell_id'] = test['sell_id']

In [ ]:
data_test.info()

In [ ]:
data_train.info()

In [ ]:
X_sub = data_test.drop(['sample','price'], axis=1)

Price and outliers

In [ ]:
data_train.price.describe()

In [ ]:
#checking for outliers in price
median = data_train.price.median()
IQR = data_train.price.quantile(0.75) - data_train.price.quantile(0.25)
perc25 = data_train.price.quantile(0.25)
perc75 = data_train.price.quantile(0.75)
print('25-й percentile: {},'.format(perc25), '75-й percentile: {},'.format(perc75)
      , "IQR: {}, ".format(IQR),"Outliers range: [{f}, {l}].".format(f=perc25 - 1.5*IQR, l=perc75 + 1.5*IQR))

In [ ]:
data_train = data_train[data_train['price']<=3175000]

In [ ]:
data_train.nunique()

CatBoost

In [ ]:
X = data_train.drop('price',axis = 1 )
X_sub = data_test.drop(['sample','price'], axis=1)

In [ ]:
X_train.info()

In [ ]:
y = data_train['price']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, 
                                                    shuffle=True, random_state=RANDOM_SEED)

In [ ]:
cat_features_ids = [1,2,3,4,5,6,7,9,10,11,12,14]

In [ ]:
X_train.info()

In [ ]:
model = CatBoostRegressor(iterations = 2000,
                          learning_rate = 0.1,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE']
                         )
model.fit(X_train, y_train,
         cat_features=cat_features_ids,
         eval_set=(X_test, y_test),
         verbose_eval=100,
         use_best_model=True,
         plot=True
         )

In [ ]:
def mape(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
predict = model.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict)):0.2f}%")

In [ ]:
predict_sub = model.predict(X_sub)
submission = data_test[['sell_id']]
submission['price'] = predict_sub

In [ ]:
submission.to_csv('submission_catboost_no_outliers.csv', index=False)

GradientBoostingRegressor

In [ ]:
gb = GradientBoostingRegressor(min_samples_split=2, learning_rate=0.03, 
                               max_depth=10, n_estimators=1000)

In [ ]:
X.info()

In [ ]:
for column in ['brand','bodyType','color','fuelType','numberOfDoors','vehicleConfiguration', 'engineDisplacement',
            'enginePower','mileage','Drive','Owners','model']:
    X[column] = X[column].astype('category').cat.codes

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, 
                                                    shuffle=True, random_state=RANDOM_SEED)

In [ ]:
gb.fit(X_train,y_train)

In [ ]:
predict = gb.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict)):0.2f}%")

In [ ]:
#preparing test dataset for submission:
for column in ['brand','bodyType','color','fuelType','numberOfDoors','vehicleConfiguration', 'engineDisplacement',
            'enginePower','mileage','Drive','Owners','model']:
    X_sub[column] = X_sub[column].astype('category').cat.codes

In [ ]:
predict_sub_gb = gb.predict(X_sub)
submission = data_test[['sell_id']]
submission['price'] = predict_sub_gb

In [ ]:
submission.to_csv('submission_GB_no_outliers.csv', index=False)

In [ ]:
submission

RandomForestRegressor

In [ ]:
reg = RandomForestRegressor(n_estimators=1000, n_jobs = -1, max_depth = 15,
                            max_features = 'log2', random_state = 42, oob_score=True,) 

In [ ]:
for column in ['brand','bodyType','color','fuelType','numberOfDoors','vehicleConfiguration', 'engineDisplacement',
            'enginePower','mileage','Drive','Owners','model']:
    data[column] = data[column].astype('category').cat.codes

In [ ]:
data_train = data[data['sample']==1]
data_test = data[data['sample']==0]

In [ ]:
X = data_train.drop(['price','sample'], axis = 1 )
X_sub = data_test.drop(['sample','price'], axis=1)

In [ ]:
y = data_train['price']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, 
                                                    shuffle=True, random_state=RANDOM_SEED)

In [ ]:
reg.fit(X_train,y_train)

In [ ]:
predict = reg.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict)):0.2f}%")

In [ ]:
data_test.info()

In [ ]:
test = pd.read_csv(DIR_TEST+'test.csv')
data_test['sell_id'] = test['sell_id']

In [ ]:
predict = reg.predict(X_sub)
submission = data_test[['sell_id']]
submission['price'] = predict

In [ ]:
submission

In [ ]:
submission.to_csv('submission_RF_no_outliers.csv', index=False)